In [ ]:
import os
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from paths import path_samoc
from timeseries import IterateOutputCESM
from ab_derivation_SST import DeriveSST as DS

## 1. generating full SST fields
### annual: from yearly TEMP_PD averaged data

In [ ]:
%%time
# 10 mins for ctrl
for run in ['ctrl', 'lpd', 'had']:
    try:
        fn = f'{path_samoc}/SST/SST_yrly_{run}.nc'
        assert os.path.exists(fn)
        print(f'file exists: {fn}')
    except:
        DS().generate_yrly_SST_files(run)  

### monthly: from monthly averaged model output

In [ ]:
# %%time
# DS.generate_monthly_SST_files('ctrl')

In [ ]:
# %%time
# # 22 sec for ctrl, 11 sec for lpd
# DS.generate_monthly_mock_linear_GMST_files('lpd')

## 2. detrending/deseasonalizing
### full SST time series

In [ ]:
%%time
# ctrl: 2:40 mins for 149 years
# lpd: < 1 min for 149 years
# had: 4:38 mins for both single and two factor detrending
for run in ['ctrl', 'lpd']:
    print(run)
    try:
        fn = f'{path_samoc}/SST/SST_GMST_sldt_yrly_{run}.nc'
        assert os.path.exists(fn)
        print(f'  file exists: {fn}')
    except:
        DS().SST_remove_forced_signal(run=run, tres='yrly', detrend_signal='GMST', time_slice='full')
    
print('had')
try:
    for dt in ['sfdt', 'tfdt']:
        fn = f'{path_samoc}/SST/SST_GMST_{dt}_yrly_had.nc'
        assert os.path.exists(fn)
        print(f'  file exists: {fn}')
except:
    DS().SST_remove_forced_signal(run='had', tres='yrly', detrend_signal='GMST', time_slice='full')

### loop over 149 year long segments 10 years apart

In [ ]:
# starting years of 149 year long segments
ctrl_starts = np.arange(1, 152, 10)
lpd_starts = np.arange(154, 415, 10)

In [ ]:
for t in ctrl_starts:
#     print(t, t+148)
    try:
        fn = f'{path_samoc}/SST/SST_GMST_sldt_yrly_ctrl_{t}_{t+148}.nc'
        assert os.path.exists(fn)
        print(f'file exists: {fn}')
    except:
        DS().SST_remove_forced_signal(run='ctrl', tres='yrly', detrend_signal='GMST', time_slice=(t,t+148))

In [ ]:
for t in lpd_starts:
#     print(t, t+148)
    try:
        fn = f'{path_samoc}/SST/SST_GMST_sldt_yrly_lpd_{t}_{t+148}.nc'
        assert os.path.exists(fn)
        print(f'file exists: {fn}')
    except:
        DS().SST_remove_forced_signal(run='lpd', tres='yrly', detrend_signal='GMST', time_slice=(t,t+148))

## 3. derive raw SST indices

In [ ]:
from bd_analysis_indices import AnalyzeIndex as AI

### full time series

In [ ]:
%%time
# ca. 1:10 min for single ctrl run, 8:45 for all
# 11 seconds for lpd and had combined
for run in ['had',  'lpd', 'ctrl']:
    if run in ['ctrl', 'lpd']:  dt = 'sldt'
    elif run=='rcp':            dt = 'sqdt'
    elif run=='had':            dt = 'tfdt'  # 'sfdt'
    
    try:
        for idx in ['AMO', 'SOM', 'TPI1', 'TPI2', 'TPI3']:
            fn = f'{path_samoc}/SST/{idx}_GMST_{dt}_raw_{run}.nc'
            assert os.path.exists(fn)
        print(f'raw index files for {run} exist')
    except:
        AI().derive_all_SST_avg_indices(run, 'full')

### segments

In [ ]:
for i, run in enumerate(['ctrl', 'lpd']):
    for t in [ctrl_starts, lpd_starts][i]:
        tslice = (t, t+148)
        try:
            for idx in ['AMO', 'SOM', 'TPI1', 'TPI2', 'TPI3']:
                fn = f'{path_samoc}/SST/{idx}_GMST_sldt_raw_{run}_{tslice[0]}_{tslice[1]}.nc'
                assert os.path.exists(fn)
            print(f'raw index files for {run} of segment {tslice} exist')
        except:
            AI().derive_all_SST_avg_indices(run, tslice)

## 4. filter SST indices

In [ ]:

AI().derive_final_SST_indices()

## autocorrelation fields

In [ ]:
%%time
# 5:40 mins for all
for run in ['ctrl', 'lpd', 'had']:
    AI().derive_yrly_autocorrelations(run)

## regression files

In [ ]:
%%time
# 13 sec for all lpd
for run in ['ctrl']:#'ctrl', 'lpd', 'had']:
    for idx in ['AMO', 'SOM', 'TPI']:
        AI().make_yrly_regression_files(run, idx)